Подход 1: градиентный бустинг "в лоб"

In [1]:
import pandas
# 1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 
features = pandas.read_csv('features.csv', index_col='match_id')
features_test = pandas.read_csv('features_test.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [2]:
#Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующиев тестовой выборке).:
y = features['radiant_win'].to_frame()
#Итог матча: duration: длительность; radiant_win: 1, если победила команда Radiant, 0 — иначе; 
#Состояние башен и барраков к концу матча: tower_status_radiant; tower_status_dire;barracks_status_radiant; barracks_status_dire
features.drop(['duration',
               'radiant_win',
               'tower_status_radiant', 
               'tower_status_dire', 
               'barracks_status_radiant',
               'barracks_status_dire'], axis=1, inplace=True)
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [3]:
# 2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. 
def check_missing(features):
    list_of_columns=list(features)
    num_rows=len(features)
    print ("N_features %.f, N_rows %.f" % (len(list_of_columns),num_rows))

    list_of_incomplete=[]
    print ("Missing: ")
    for i in range(0,len(list_of_columns)-1):
        i_count = features[list_of_columns[i]].count()
        if i_count!=num_rows:
            print ("%s , %.f " % (list_of_columns[i],100-i_count/num_rows*100))
            list_of_incomplete.append((i,list_of_columns[i],100-i_count/num_rows*100))
#list_of_columns[len(list_of_columns)-1]
    return list_of_incomplete


In [4]:
list_initial = check_missing(features)

N_features 102, N_rows 97230
Missing: 
first_blood_time , 20 
first_blood_team , 20 
first_blood_player1 , 20 
first_blood_player2 , 45 
radiant_bottle_time , 16 
radiant_courier_time , 1 
radiant_flying_courier_time , 28 
radiant_first_ward_time , 2 
dire_bottle_time , 17 
dire_courier_time , 1 
dire_flying_courier_time , 27 


In [5]:
#Много ли пропусков в данных? 
len(list_initial)

list_initial.sort(key=lambda x:x[2])
list_initial

[(95, 'dire_courier_time', 0.6952586650210861),
 (87, 'radiant_courier_time', 0.7117144914121241),
 (93, 'radiant_first_ward_time', 1.8883060783708743),
 (86, 'radiant_bottle_time', 16.138023243854775),
 (94, 'dire_bottle_time', 16.602900339401415),
 (82, 'first_blood_time', 20.110048338990026),
 (83, 'first_blood_team', 20.110048338990026),
 (84, 'first_blood_player1', 20.110048338990026),
 (96, 'dire_flying_courier_time', 26.841509822071373),
 (88, 'radiant_flying_courier_time', 28.261853337447292),
 (85, 'first_blood_player2', 45.240152216394115)]

In [6]:
# 3. Замените пропуски на нули с помощью функции fillna(). 
features_clean = features.fillna(0)
#Check:
list_of_mis_after_clean = check_missing(features_clean)

N_features 102, N_rows 97230
Missing: 


In [7]:
# 4. Какой столбец содержит целевую переменную? Запишите его название.
list(y)


['radiant_win']

In [8]:
# 5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг
# над деревьями на имеющейся матрице "объекты-признаки". 
#Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), 
#не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице 
#отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами 
#при оценивании качества. 
from sklearn.cross_validation import KFold,cross_val_score
kf = KFold(y.size, n_folds=5, shuffle=True, random_state=241)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
#Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной 
#кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте 
#следующие значения для количества деревьев: 10, 20, 30). 

#Долго ли настраивались классификаторы? 
import time
import datetime
start_time = datetime.datetime.now()
time.sleep(3) # вместо этой строчки разместить замеряемый код
print ('Time elapsed:', datetime.datetime.now() - start_time)


Time elapsed: 0:00:03.001411


In [10]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
gb_qual = []
n_trees = [10, 20, 30, 50, 70,100,150,200]
Y=np.array(y)
for i in n_trees:
    gb = GradientBoostingClassifier(n_estimators=i, random_state=241)
    start_time = datetime.datetime.now()
    gb_scores = cross_val_score(gb, features_clean, Y.ravel(), cv=kf, scoring='roc_auc')
    run_time=datetime.datetime.now() - start_time
    print ('Time elapsed:', run_time)
    print (gb_scores)
    gb_qual.append((i,run_time,np.mean(gb_scores)))

Time elapsed: 0:00:19.296906
[0.66943496 0.65627754 0.66390454 0.66281223 0.66950933]
Time elapsed: 0:00:35.280699
[0.68939011 0.67567277 0.68201026 0.67977584 0.68741889]
Time elapsed: 0:00:56.224454
[0.69641668 0.68365441 0.68731862 0.68728458 0.69280674]
Time elapsed: 0:01:23.578763
[0.70382382 0.69325852 0.69452532 0.69565719 0.7000093 ]
Time elapsed: 0:01:55.002650
[0.70784114 0.69833267 0.6990506  0.70019753 0.70473656]
Time elapsed: 0:02:43.064857
[0.71211028 0.70275727 0.70340522 0.70450889 0.70884943]
Time elapsed: 0:04:06.177800
[0.71546705 0.70731614 0.70793497 0.70887821 0.71252658]
Time elapsed: 0:05:23.668560
[0.71884063 0.71030142 0.71135838 0.71139223 0.71574489]


In [11]:
#Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, 
#скорее всего, продолжит расти при дальнейшем его увеличении?
gb_qual

[(10, datetime.timedelta(0, 19, 296906), 0.6643877206345742),
 (20, datetime.timedelta(0, 35, 280699), 0.6828535735340822),
 (30, datetime.timedelta(0, 56, 224454), 0.6894962060591201),
 (50, datetime.timedelta(0, 83, 578763), 0.6974548316948366),
 (70, datetime.timedelta(0, 115, 2650), 0.7020316980434622),
 (100, datetime.timedelta(0, 163, 64857), 0.7063262181631453),
 (150, datetime.timedelta(0, 246, 177800), 0.7104245908535362),
 (200, datetime.timedelta(0, 323, 668560), 0.713527508807472)]

Report

Q1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
A1.:
1. first_blood_player2 (второй игрок, причастный к событию) - в 55% случаев только 1 игрок причастен к событию;
2. radiant_flying_courier_time (время приобретения предмета "flying_courier") - в 72% случаев этот предмет не покупается

Q2. Как называется столбец, содержащий целевую переменную?
A2.:
['radiant_win']

Q3.1 Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? 
Q3.2 Какое качество при этом получилось?
A3.1: для 30 деревьев 56 секунд, 
A3.2: качество = 0.69

Q4.1 Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? 
Q4.2 Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
A4.1: Нет, после 30 деревьев качество можно увеличить до 71% на 200 деревьях, но лучше стоит подобрать более оптимальный алгоритм
A4.2: Можно обучать на подвыборках, либо уменьшить глубину

In [12]:
gb = GradientBoostingClassifier(n_estimators=250, random_state=241)
gb.fit(features_clean, Y.ravel())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=250,
              presort='auto', random_state=241, subsample=1.0, verbose=0,
              warm_start=False)

In [13]:
features_test_clean = features_test.fillna(0)
gb.predict_proba(features_test_clean)[:,1]

array([0.45677799, 0.69648984, 0.41123343, ..., 0.2297639 , 0.34162075,
       0.58538676])